In [11]:
import os 
import pandas as pd
from PIL import Image
from io import BytesIO
import math
import base64
import torch
from transformers import AutoImageProcessor, ResNetForImageClassification
from qdrant_client import QdrantClient, models
from qdrant_client.models import VectorParams, Distance
from dotenv import load_dotenv

load_dotenv()
qclient = QdrantClient(url=os.getenv("QDRANT_URL"), api_key=os.getenv("QDRANT_API_KEY"))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
# Fetch all images

base_directory = "./images/animals/train/dog"
all_image_urls = os.listdir(base_directory)
all_image_urls[:10]

['dog1866.jpg',
 'dog1635.jpg',
 'dog2215.jpg',
 'dog1576.jpg',
 'dog815.jpg',
 'dog1993.jpg',
 'dog566.jpg',
 'dog1303.jpg',
 'dog942.jpg',
 'dog1336.jpg']

In [13]:
# Load images

sample_image_urls = list(map(lambda x: os.path.join(base_directory, x), all_image_urls))
sample_image_urls


['./images/animals/train/dog/dog1866.jpg',
 './images/animals/train/dog/dog1635.jpg',
 './images/animals/train/dog/dog2215.jpg',
 './images/animals/train/dog/dog1576.jpg',
 './images/animals/train/dog/dog815.jpg',
 './images/animals/train/dog/dog1993.jpg',
 './images/animals/train/dog/dog566.jpg',
 './images/animals/train/dog/dog1303.jpg',
 './images/animals/train/dog/dog942.jpg',
 './images/animals/train/dog/dog1336.jpg',
 './images/animals/train/dog/dog55.jpg',
 './images/animals/train/dog/dog24.jpg',
 './images/animals/train/dog/dog2344.jpg',
 './images/animals/train/dog/dog2511.jpg',
 './images/animals/train/dog/dog2605.jpg',
 './images/animals/train/dog/dog2484.jpg',
 './images/animals/train/dog/dog1198.jpg',
 './images/animals/train/dog/dog2500.jpg',
 './images/animals/train/dog/dog171.jpg',
 './images/animals/train/dog/dog1815.jpg',
 './images/animals/train/dog/dog2117.jpg',
 './images/animals/train/dog/dog465.jpg',
 './images/animals/train/dog/dog1856.jpg',
 './images/animals/t

In [14]:
# Create dataframe to store image metadata

metada_df = pd.DataFrame.from_records({"image_url": sample_image_urls})
metada_df["type"] = "dog"
metada_df

,image_url,type
0,./images/animals/train/dog/dog1866.jpg,dog
1,./images/animals/train/dog/dog1635.jpg,dog
2,./images/animals/train/dog/dog2215.jpg,dog
3,./images/animals/train/dog/dog1576.jpg,dog
4,./images/animals/train/dog/dog815.jpg,dog
...,...,...
2622,./images/animals/train/dog/dog2472.jpg,dog
2623,./images/animals/train/dog/dog2211.jpg,dog
2624,./images/animals/train/dog/dog384.jpg,dog
2625,./images/animals/train/dog/dog1867.jpg,dog


In [15]:
# Create PIL images from the local urls

images = list(map(lambda x: Image.open(x), metada_df["image_url"]))
images

[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512>,
 <PIL.Jpeg

In [16]:
# Represent images as base64 encoded strings to be stored in Qdrant and save storage

target_width = 256

def resize_image(image_url):
    image = Image.open(image_url)
    image_aspect_ratio = image.width / image.height
    resized_image = image.resize([target_width, math.floor(target_width / image_aspect_ratio)])

    return resized_image

def image_to_base64(image):
    buffered = BytesIO()
    image.save(buffered, format="JPEG")
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")

    return img_str


resized_images = list(map(lambda x: resize_image(x), sample_image_urls))
base64_images = list(map(lambda x: image_to_base64(x), resized_images))
metada_df["base64"] = base64_images
metada_df

,image_url,type,base64
0,./images/animals/train/dog/dog1866.jpg,dog,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
1,./images/animals/train/dog/dog1635.jpg,dog,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
2,./images/animals/train/dog/dog2215.jpg,dog,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
3,./images/animals/train/dog/dog1576.jpg,dog,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
4,./images/animals/train/dog/dog815.jpg,dog,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
...,...,...,...
2622,./images/animals/train/dog/dog2472.jpg,dog,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
2623,./images/animals/train/dog/dog2211.jpg,dog,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
2624,./images/animals/train/dog/dog384.jpg,dog,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
2625,./images/animals/train/dog/dog1867.jpg,dog,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...


In [17]:
# Create embeddings for the images

# Load processor and model
processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50", use_fast=True)
model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50").to(device)

# Process images in smaller batches
batch_size = 8  # Adjust batch size as needed
embeddings_list = []

for i in range(0, len(images), batch_size):
    batch_images = images[i:i+batch_size]
    inputs = processor(batch_images, return_tensors="pt", use_fast_tokenizer=False).to(device)
    outputs = model(**inputs)
    embeddings_list.append(outputs.logits.cpu().detach())

# Combine embeddings
embeddings = torch.cat(embeddings_list, dim=0)
embeddings

/home/alberto/.pyenv/versions/qdrant_test/lib/python3.9/site-packages/transformers/image_processing_utils.py:41: UserWarning: The following named arguments are not valid for `ConvNextImageProcessor.preprocess` and were ignored: 'use_fast_tokenizer'
  return self.preprocess(images, **kwargs)


tensor([[ -9.2493, -10.0374, -10.1949,  ...,  -8.9388,  -7.5086,  -8.6677],
        [-10.3305, -11.4099,  -9.6981,  ..., -12.1317,  -9.3344,  -9.7125],
        [ -8.3917, -10.0928, -10.8491,  ...,  -9.5784,  -8.2267,  -9.3161],
        ...,
        [-10.0213,  -9.2212, -10.5616,  ..., -10.5752,  -7.9525,  -8.7576],
        [ -9.8496, -10.0211, -10.8486,  ..., -10.3832,  -7.2914,  -7.3592],
        [-11.7452,  -9.9602, -10.6364,  ..., -11.5012,  -9.5002,  -8.7100]])

In [18]:
embeddings_length = len(embeddings[0])
embeddings_length

1000

In [19]:
# Create collection in Qdrant

collection_name = "dog_images"
collection = qclient.create_collection(collection_name=collection_name, 
                                       vectors_config=VectorParams(
                                           size=embeddings_length, 
                                           distance=Distance.COSINE
                                           )
                                        )

collection

True

In [20]:
# Transform the dataframes to dictionaries

metadata_dicts = metada_df.to_dict(orient="records")
metadata_dicts[:5]

[{'image_url': './images/animals/train/dog/dog1866.jpg',
  'type': 'dog',
  'base64': '/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjL/wAARCAEAAQADASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwC9PJDpWqOYSDbudjqDxjPXFdVol35WoyRuAomX5hnjP/1xXnPiiwSZPt1skgVTklTnr9Kz9F8RXFpOskNwz

In [21]:
records = [
    models.Record(
        id=x,
        vector=embeddings[x],
        payload=metadata_dicts[x]
    )
    for x, _ in enumerate(metadata_dicts)]

qclient.upload_records(collection_name=collection_name, records=records)

/tmp/ipykernel_2077/2271203817.py:9: DeprecationWarning: `upload_records` is deprecated, use `upload_points` instead
  qclient.upload_records(collection_name=collection_name, records=records)
